# Parsing Tabular Data

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse tabular files (CSV) files similar to the format established in
the [RTS-GMLC](github.com/gridmod/rts-gmlc/RTS_Data/SourceData) and create a `System` using
[PowerSystems.jl](github.com/NREL-SIIP/PowerSystems.jl)

### Dependencies

In [1]:
using PowerSystems
using TimeSeries
using Dates

### Fetch Data
PowerSystems.jl links to some test data that is suitable for this example.
Let's download the test data

In [2]:
PowerSystems.download(PowerSystems.TestData; branch = "master") # *note* add `force=true` to get a fresh copy
base_dir = dirname(dirname(pathof(PowerSystems)));

### The tabular data format relies on a folder containing `*.csv` files and a `user_descriptors.yaml` file
First, we'll read the tabular data

In [3]:
RTS_GMLC_DIR = joinpath(base_dir, "data", "RTS_GMLC");
rawsys = PowerSystems.PowerSystemTableData(
    RTS_GMLC_DIR,
    100.0,
    joinpath(RTS_GMLC_DIR, "user_descriptors.yaml"),
    timeseries_metadata_file = joinpath(RTS_GMLC_DIR, "timeseries_pointers.json"),
    generator_mapping_file = joinpath(RTS_GMLC_DIR, "generator_mapping_multi_start.yaml"),
)

[ Info: Parsing csv files in Reserves ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Down.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Flex_Up.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Down.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Reg_Up.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R1.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R2.csv ...
[ Info: Parsing csv data in DAY_AHEAD_regional_Spin_Up_R3.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Reg_Down.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Reg_Up.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R1.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R2.csv ...
[ Info: Parsing csv data in REAL_TIME_regional_Spin_Up_R3.csv ...
[ Info: Successfully parsed Reserves
[ Info: Parsing csv data in branch.csv ...
[ Info: Successfully parsed branch.csv
[ Info: Parsing csv data in bus.csv ...
[ Info: Successfully pa

PowerSystems.PowerSystemTableData:
  directory:  /home/runner/.julia/packages/PowerSystems/gGFFl/data/RTS_GMLC
  timeseries_metadata_file:  /home/runner/.julia/packages/PowerSystems/gGFFl/data/RTS_GMLC/timeseries_pointers.json
  base_power:  100.0
  PowerSystems.InputCategoryModule.InputCategory.RESERVE = 6:  7×7 DataFrame
  PowerSystems.InputCategoryModule.InputCategory.STORAGE = 8:  22×8 DataFrame
  PowerSystems.InputCategoryModule.InputCategory.BRANCH = 1:  120×14 DataFrame
  PowerSystems.InputCategoryModule.InputCategory.GENERATOR = 4:  158×58 DataFrame
  PowerSystems.InputCategoryModule.InputCategory.BUS = 2:  73×15 DataFrame
  PowerSystems.InputCategoryModule.InputCategory.DC_BRANCH = 3:  1×60 DataFrame


### Create a `System`
Next, we'll create a `System` from the `rawsys` data. Since a `System` is predicated on a
time series resolution and the `rawsys` data includes both 5-minute and 1-hour resolution
time series, we also need to specify which time series we want to include in the `System`.
The `time_series_resolution` kwarg filters to only include time series with a matching resolution.

In [4]:
sys = System(rawsys; time_series_resolution = Dates.Hour(1));
horizon = 24;
interval = Dates.Hour(24);
transform_single_time_series!(sys, horizon, interval);
sys

[ Info: Unit System changed to InfrastructureSystems.UnitSystemModule.UnitSystem.DEVICE_BASE = 1
┌ Warning: Missing PowerSystems.InputCategoryModule.InputCategory.LOAD = 5 data.
└ @ PowerSystems ~/.julia/packages/PowerSystems/gGFFl/src/parsers/power_system_table_data.jl:213
┌ Warning: User-defined column name Startup Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems ~/.julia/packages/PowerSystems/gGFFl/src/parsers/power_system_table_data.jl:1376
┌ Warning: User-defined column name Shutdown Ramp Rate MW/min is not in dataframe.
└ @ PowerSystems ~/.julia/packages/PowerSystems/gGFFl/src/parsers/power_system_table_data.jl:1376
┌ Warning: User-defined column name Status at Start is not in dataframe.
└ @ PowerSystems ~/.julia/packages/PowerSystems/gGFFl/src/parsers/power_system_table_data.jl:1376
┌ Warning: User-defined column name Time at Status is not in dataframe.
└ @ PowerSystems ~/.julia/packages/PowerSystems/gGFFl/src/parsers/power_system_table_data.jl:1376
┌ Warning: User-defined

System
┌───────────────────┬─────────────┐
│ Property          │ Value       │
├───────────────────┼─────────────┤
│ System Units Base │ DEVICE_BASE │
│ Base Power        │ 100.0       │
│ Base Frequency    │ 60.0        │
│ Num Components    │ 525         │
└───────────────────┴─────────────┘

Static Components
┌───────────────────────────────────────────┬───────┬────────────────────────┬──
│ Type                                      │ Count │ Has Static Time Series │ ⋯
├───────────────────────────────────────────┼───────┼────────────────────────┼──
│ Arc                                       │ 109   │ false                  │ ⋯
│ Area                                      │ 3     │ false                  │ ⋯
│ Bus                                       │ 73    │ false                  │ ⋯
│ GenericBattery                            │ 1     │ false                  │ ⋯
│ HVDCLine                                  │ 1     │ false                  │ ⋯
│ HydroDispatch                             │ 20    │ true                   │ ⋯
│ Line                                      │ 105   │ false                  │ ⋯
│ LoadZone                                  │ 3     │ true                   │ ⋯
│ PowerLoad                                 │ 51    │ true                   │ ⋯
│ RenewableDispatch                         │ 30    │ true                   │ ⋯
│ RenewableFix                              │ 31    │ true                   │ ⋯
│ TapTransformer                            │ 15    │ false                  │ ⋯
│ ThermalMultiStart                         │ 76    │ false                  │ ⋯
│ VariableReserve{PowerSystems.ReserveDown} │ 2     │ true                   │ ⋯
│ VariableReserve{PowerSystems.ReserveUp}   │ 5     │ true                   │ ⋯
└───────────────────────────────────────────┴───────┴────────────────────────┴──
                                                                1 column omitted

Time Series Summary
┌──────────────────────────────────┬─────────────────────┐
│ Property                         │ Value               │
├──────────────────────────────────┼─────────────────────┤
│ Components with time series data │ 142                 │
│ Total StaticTimeSeries           │ 182                 │
│ Total Forecasts                  │ 182                 │
│ Resolution                       │ 60 minutes          │
│ First initial time               │ 2020-01-01T00:00:00 │
│ Last initial time                │ 2020-01-01T00:00:00 │
│ Horizon                          │ 24                  │
│ Interval                         │ 0 minutes           │
│ Forecast window count            │ 1                   │
└──────────────────────────────────┴─────────────────────┘

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*